In [1]:
%pip install bs4 selenium webdriver_manager
%pip install python-dotenv
%pip install haversine

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=9e08a9cf5e894db54c2f5b591a22f3b942d59920ce8e9071d363bff28ebd3c81
  Stored in directory: c:\users\lucet\appdata\local\pip\cache\wheels\75\78\21\68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

df_bus = pd.read_csv('bus.csv', encoding='euc-kr')
df_subway = pd.read_csv('subway.csv', encoding='euc-kr')
df_bike = pd.read_csv('bike.csv', encoding='cp949')

In [4]:
df_bus

,ARS-ID,정류소명,X좌표,Y좌표
0,1001,종로2가사거리,126.987750,37.569765
1,1002,창경궁.서울대학교병원,126.996566,37.579183
2,1003,명륜3가.성대입구,126.998340,37.582671
3,1004,종로2가.삼일교,126.987613,37.568579
4,1005,혜화동로터리.여운형활동터,127.001744,37.586243
...,...,...,...,...
11310,25995,우성아파트,127.139338,37.550386
11311,25996,우성아파트,127.140046,37.550643
11312,25997,조일약국,127.123596,37.533630
11313,25998,성내시장,127.125497,37.536155


In [5]:
df_subway[['code', 'name', 'lat(y)', 'lng']]

,code,name,lat(y),lng
0,1908.0,녹양,37.759380,127.042292(x)
1,1002.0,남영,37.541021,126.9713
2,1003.0,용산,37.529849,126.964561
3,1004.0,노량진,37.514219,126.942454
4,1005.0,대방,37.513342,126.926382
...,...,...,...,...
725,3135.0,지식정보단지,37.378384,126.645168
726,3136.0,인천대입구,37.386007,126.639484
727,3137.0,센트럴파크,37.393054,126.634729
728,3138.0,국제업무지구,37.399907,126.630347


In [6]:
df_bike[['대여소\n번호', "보관소(대여소)명", "위도(Y)", "경도(X)"]]

,대여소\n번호,보관소(대여소)명,위도(Y),경도(X)
0,2541,옛골 입구,37.430977,127.070335
1,1861,IBK기업은행 석수역 지점 앞,37.436981,126.903297
2,1858,석수역1번출구 앞 (SK주유소),37.437271,126.902687
3,1820,신한은행 시흥대로금융센터지점,37.443340,126.903358
4,1816,금천폭포공원 앞,37.446861,126.903831
...,...,...,...,...
2581,1731,도봉고등학교 맞은편,37.687630,127.042213
2582,1750,창포원 남쪽 입구,37.688267,127.048897
2583,1672,상계동수락리버시티,37.688599,127.053406
2584,1703,도봉산광역환승센터앞,37.689720,127.045197


In [22]:
import time
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

class KakaoRouteFinder:
    def __init__(self):
        self.driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    def fetch_bike(self, verbose=True, init=True, time_delta=0.2):
        if init:
            self.driver.find_element(by=By.CSS_SELECTOR, value=f"#biketab").click()
            time.sleep(time_delta)
            
        if verbose:
            print(" > bike")
            self.driver.find_element(by=By.CSS_SELECTOR, value=f"div.BikeRouteResultView > ul > li:nth-child(1)").click()        
            time.sleep(time_delta)
            self.driver.find_element(by=By.CSS_SELECTOR, value=f"div.BikeRouteResultView > ul > li:nth-child(2)").click()        
            time.sleep(time_delta)
            self.driver.find_element(by=By.CSS_SELECTOR, value=f"div.BikeRouteResultView > ul > li:nth-child(3)").click()        
            time.sleep(time_delta)

        element = self.driver.find_element(by=By.CSS_SELECTOR, value=f"div.BikeRouteResultView")
        return element.get_attribute('innerHTML')
    
    def fetch_walk(self, verbose=True, init=True, time_delta=0.2):
        if init:
            self.driver.find_element(by=By.CSS_SELECTOR, value=f"#walktab").click()
            time.sleep(time_delta)

        if verbose:
            print(" > walk")

        element = self.driver.find_element(by=By.CSS_SELECTOR, value=f"div.WalkRouteResultView")
        return element.get_attribute('innerHTML')
        
    def fetch_transit(self, verbose=True, init=True, time_delta=0.2):    
        if init:
            self.driver.find_element(by=By.CSS_SELECTOR, value=f"#transittab").click()
            time.sleep(2)

        if verbose:
            print(" > transit")
            for route in self.driver.find_elements(by=By.CSS_SELECTOR, value=f"li.TransitRouteItem"):
                route.click()
                for more_route in route.find_elements(by=By.CSS_SELECTOR, value=f"span.moreBtn"):
                    more_route.click()
                    time.sleep(time_delta)
                time.sleep(time_delta)

        element = self.driver.find_element(by=By.CSS_SELECTOR, value=f"ul.TransitTotalPanel")
        return element.get_attribute('innerHTML')

    def find_routes_congnamul(self, type, origin, dest, verbose=False, init=False, time_delta=0.2):
        (org_x, org_y) = origin
        (des_x, des_y) = dest
        base_url = f"https://map.kakao.com/?map_type=TYPE_MAP&target={type}&rt={int(org_x)},{int(org_y)},{int(des_x)},{int(des_y)}&rt1=ORIGIN&rt2=DESTINATION"
        self.driver.get(base_url)
        time.sleep(time_delta)

        self.driver.find_element(by=By.CSS_SELECTOR, value="body").click()
        time.sleep(time_delta)
        
        try:
            dimmed_layer_selector = "#dimmedLayer"
            self.driver.find_element(
                by=By.CSS_SELECTOR,
            value=dimmed_layer_selector
            ).click()
        except ElementNotInteractableException:
            pass

        try:
            if (type == 'bike'):
                return self.fetch_bike(verbose, init)
            else:
                return None
        except Exception as e:
            print(e)

    def find_routes(self, origin, dest, time_delta=0.2):
        print(origin, dest)

        base_url = f"https://map.kakao.com/?map_type=TYPE_MAP&target=bike&sName={origin}&eName={dest}"
        self.driver.get(base_url)
        time.sleep(time_delta)

        self.driver.find_element(by=By.CSS_SELECTOR, value="body").click()
        time.sleep(time_delta)
        
        try:
            dimmed_layer_selector = "#dimmedLayer"
            self.driver.find_element(
                by=By.CSS_SELECTOR,
            value=dimmed_layer_selector
            ).click()
        except ElementNotInteractableException:
            pass

        try:
            return {
                # 'bike': fetch_bike(),
                # 'walk': fetch_walk(),
                'transit': self.fetch_transit()
            }
        except Exception as e:
            print(e)

    def __del__(self):
        self.driver.close()

In [8]:
# org, des = ('서울시청', '서울고속터미널')
org, des = ('대한건축학회', '서울대학교 자하연')
routes = KakaoRouteFinder().find_routes(org, des)



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\lucet\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache


대한건축학회 서울대학교 자하연
 > transit


### Bike

In [41]:
# soup = BeautifulSoup(routes['bike'], 'html.parser')
# for route in (soup.find_all('li', {"class": "BikeRouteItem"})):
#     res = {
#         "mode": route.find('span', {"class": "mode"}).text,
#         "time": route.find('span', {"class": "time"}).text.strip(),
#         "distance": route.find('span', {"class": "distance"}).text,
#         "altitude": route.find('span', {"class": "altitude"}).text,
#         "calories": route.find('span', {"class": "calories"}).text,
#     }
#     print(res)

### Transit

In [42]:
# soup = BeautifulSoup(routes['transit'], 'html.parser')
# for route in (soup.find_all('li', {"class": "TransitRouteItem"})):
#     res = {
#         "time": route.find('span', {"class": "time"}).text.strip(),
#         "info": route.find('span', {"class": "walkTime"}).get('title'),
#     }
#     print(res)

### Walk

In [43]:
# soup = BeautifulSoup(routes['walk'], 'html.parser')
# for route in (soup.find_all('li', {"class": "WalkRouteItem"})):
#     res = {
#         "mode": route.find('span', {"class": "mode"}).text,
#         "time": route.find('span', {"class": "time"}).text.strip(),
#         "info": route.find('div', {"class": "info"}).text.strip(),
#     }
#     print(res)

### Coordinate Transform

In [9]:
import requests
import dotenv
from os import environ

dotenv.load_dotenv('.env', override=True)

def transform(coord, input_coord='WCONGNAMUL', output_coord='WGS84'):
    URL = f'https://dapi.kakao.com/v2/local/geo/transcoord.json?x={coord[0]}&y={coord[1]}&input_coord={input_coord}&output_coord={output_coord}'
    headers = {'Authorization': f'KakaoAK {environ.get("KAKAO_REST_API_KEY")}'}
    new_coord = requests.get(URL, headers=headers).json()['documents'][0]
    return (new_coord['x'], new_coord['y'])

transform((495285,1129803), 'WCONGNAMUL', 'WGS84')

(126.97865225830876, 37.56682600466712)

In [10]:
(org_x, org_y) = transform((126.97865225830876, 37.56682600466712), 'WGS84', 'WCONGNAMUL')
(des_x, des_y) = transform((127.00515186540567, 37.49275468135762), 'WGS84', 'WCONGNAMUL')
print(f'https://map.kakao.com/?map_type=TYPE_MAP&target=car&rt={int(org_x)},{int(org_y)},{int(des_x)},{int(des_y)}&rt1=ORIGIN&rt2=DESTINATION')

https://map.kakao.com/?map_type=TYPE_MAP&target=car&rt=495285,1129803,501139,1109250&rt1=ORIGIN&rt2=DESTINATION


In [11]:
soup = BeautifulSoup(routes['transit'], 'html.parser')

S = set()
for route in (soup.find_all('li', {"class": "TransitRouteItem"})):
    nodes = [name.text.strip().replace(" 승차", "").replace(" 하차", "").replace(" 정류장", "") for name in route.find_all('a', {"data-id": "name"})]
    print(nodes)
    S.update(nodes)
    nodes = [node.text.strip() for node in route.find_all('li', {"class": "nodeName"})]
    print(nodes)
    S.update(nodes)

['대한건축학회', '방배역', '서울대입구역', '서울대입구역', '자연대.행정관입구', '서울대학교 자하연']
['사당역', '낙성대역', '관악경찰서.관악소방서', '서울여상.서울문영여중고앞', '서울대학교.치과병원.동물병원', '서울대학교', '서울대정문', '법대.사회대입구']
['대한건축학회', '대우효령아파트', '낙성대입구', '낙성대역', '노천강당', '서울대학교 자하연']
['종로교회', '서울교통공사', '사당1동관악시장앞', '예술인마을', '서울미술고.인헌중고', '낙성대입구', '인헌초등학교', '낙성대현대아파트', '낙성대공원.영어마을', '서울과학전시관', '호암교수회관', '인헌아파트', '호암교수회관', '가족생활동', '서울대후문.연구공원', '학부생활관', '대학원생활관', '관악사삼거리']
['대한건축학회', '방배역', '낙성대역', '낙성대역', '노천강당', '서울대학교 자하연']
['사당역', '낙성대입구', '인헌초등학교', '낙성대현대아파트', '낙성대공원.영어마을', '서울과학전시관', '호암교수회관', '인헌아파트', '호암교수회관', '가족생활동', '서울대후문.연구공원', '학부생활관', '대학원생활관', '관악사삼거리']
['대한건축학회', '방배역', '서울대입구역', '서울대입구역', '국제대학원', '서울대학교 자하연']
['사당역', '낙성대역', '관악경찰서.관악소방서', '서울여상.서울문영여중고앞', '서울대학교.치과병원.동물병원', '서울대학교', '경영대', '수의대입구.보건대학원앞']
['대한건축학회', '대우효령아파트', '봉천사거리.봉천중앙시장', '봉천사거리.위버폴리스', '자연대.행정관입구', '서울대학교 자하연']
['종로교회', '서울교통공사', '사당1동관악시장앞', '예술인마을', '서울미술고.인헌중고', '낙성대입구', '행운동주민센터', '서울대입구역', '관악경찰서.관악소방서', '서울여상.서울문영여중고앞', '서울대학교.치과병원.동물병원', '서울대학교', '서울대

In [12]:
from haversine import haversine

def get_distance(org, des):
    return haversine(org, des, unit='km')

def get_nearest_bike(lat, lng):
    delta = 5e-3
    df = df_bike[(df_bike["위도(Y)"] > lat-delta) & (df_bike["위도(Y)"] < lat+delta) \
               & (df_bike["경도(X)"] > lng-delta) & (df_bike["경도(X)"] < lng+delta) ]

    min_bike, min_dist = "N/A", float("inf")
    for i, bike in df.iterrows():
        dist = get_distance((lat, lng), bike[["위도(Y)", "경도(X)"]])
        if min_dist > dist:
            min_dist = dist
            min_bike = bike
    return min_dist, min_bike

def get_coordinate(keyword):
    URL = f'https://dapi.kakao.com/v2/local/search/keyword.json?query={keyword}'
    headers = {'Authorization': f'KakaoAK {environ.get("KAKAO_REST_API_KEY")}'}
    coord = requests.get(URL, headers=headers).json()['documents'][0]
    return (float(coord['y']), float(coord['x']))

org_coord = get_coordinate(org)
des_coord = get_coordinate(des)

# get_nearest_bike(37.550007, 126.914825)

In [18]:
from tqdm import tqdm

seoul_bike = {}
for s in tqdm(S, desc="Retrieve Seoul Bike Stations: "):
    bus = df_bus[df_bus['정류소명'] == s]
    subway = df_subway[df_subway['name'] == s.replace('역', '')]
    try:
        if len(subway):
            lat, lng = subway[['lat(y)', 'lng']].iloc[0]
        elif len(bus):
            lat, lng = bus[['Y좌표', 'X좌표']].iloc[0]
        else:
            # print(s, 'N/A')
            continue
        _, bike = get_nearest_bike(float(lat), float(lng))
        bike_id, bike_name, bike_lat, bike_lng = bike[['대여소\n번호', '보관소(대여소)명', '위도(Y)', '경도(X)']]
        # print(s, lat, lng, bike_name)
        congnamul = transform((bike_lng, bike_lat), 'WGS84', 'WCONGNAMUL')
        seoul_bike[bike_id] = ( \
            bike_name, \
            bike_lat, \
            bike_lng, \
            get_distance(org_coord, (bike_lat, bike_lng)), \
            get_distance(des_coord, (bike_lat, bike_lng)), \
            congnamul,
        )
    except:
        # print('N/A')
        pass
# seoul_bike

Retrieve Seoul Bike Stations: 100%|██████████| 40/40 [00:03<00:00, 10.06it/s]


In [19]:
for id in seoul_bike.keys():
    name, lat, lng, org_dist, des_dist, congnamul = seoul_bike[id]
    coord = (lat, lng)
    print(id, name, coord, org_dist - des_dist, congnamul)

2129 낙성대 과학전시관 (37.46905518, 126.9581451) 2.3143396046693194 (490744.0, 1102676.0)
2104 사당역 5번출구 (37.47608948, 126.9813309) -1.8994199110750185 (495872.0, 1104626.0)
2119 중앙동 동진빌딩 (37.48425674, 126.9546967) 0.8407948704666071 (489983.0, 1106894.0)
3813 낙성대역 5번출구 (37.47752762, 126.9627686) 0.6564455118418944 (491767.0, 1105027.0)
2178 서울대학교 정문 (37.46681976, 126.9488068) 3.5038905284969166 (488678.0, 1102057.0)
2165 JK장평타워 (37.47648239, 126.9653626) 0.4320688190782569 (492341.0, 1104736.0)
3806 낙성대 교수아파트 (37.46895981, 126.9576569) 2.3884253691169786 (490636.0, 1102650.0)
2130 영어마을 관악캠프 (37.47198868, 126.9584274) 1.8799562964369896 (490806.0, 1103490.0)
2128 관악구청교차로 (37.47916412, 126.9525833) 1.5799665706116306 (489515.0, 1105482.0)
3815 광일빌딩 (37.47639847, 126.9768372) -1.2383071981719203 (494878.0, 1104712.0)
2116 에이스에이존빌딩 (37.48170471, 126.9508286) 1.4498447160018961 (489127.0, 1106187.0)
2111 서울대입구역 1번출구 (37.48086929, 126.9533157) 1.3211163595937352 (489677.0, 1105955.0)
2281 연세사랑병원신관앞

In [25]:
finder = KakaoRouteFinder()
bikes = sorted(seoul_bike.items(), key=lambda item: item[1][-3]-item[1][-2])
for i, srt in tqdm(enumerate(bikes), desc="Finding Bike Routes: "):
    (_, (srt_name, srt_lat, srt_lng, _, _, srt_coord)) = srt
    for j in range(i+1, len(bikes)):
        end = bikes[j]
        (_, (end_name, end_lat, end_lng, _, _, end_coord)) = end
        # print(i, j, srt_name, end_name, srt_coord, end_coord)
        finder.find_routes_congnamul('bike', srt_coord, end_coord, verbose=False, time_delta=0.1)
        break
    break
del finder



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [C:\Users\lucet\.wdm\drivers\chromedriver\win32\101.0.4951.41\chromedriver.exe] found in cache
Finding Bike Routes: : 0it [00:02, ?it/s]
